<a href="https://colab.research.google.com/github/CarolMartinezPerez/03MAIR-Algoritmos-de-Optimizacion-2019/blob/master/AG2/Carolina_Martinez_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Carolina Martínez</br>
Link:https://colab.research.google.com/drive/10fbw50EfQVj6NM2ohH8NhKqYbVXu6Wru </br>
AG2. Algoritmos de Optimización</br>


In [0]:
# ARRAY DE TARIFAS:
TARIFAS = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,11],
[999,999, 0,1,999,4,10],
[999,999,999, 0,5,6,9],
[999,999, 999,999,0,999,4],
[999,999, 999,999,999,0,3],
[999,999,999,999,999,999,0]
]

In [0]:
def Precios (TARIFAS):
   # Total de nodos
   N = len(TARIFAS[0])

   #Inicialización de la tabla de precios
   PRECIOS = [ [9999]*N for i in [9999]*N]
   RUTA = [ [""]*N for i in [""]*N]
  
   for i in range(N-1):
     for j in range(i+1, N):
       MIN = TARIFAS[i][j]
       RUTA[i][j] = i
      
       for k in range(i, j):
         if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
             MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j] )
             RUTA[i][j] = k
         PRECIOS[i][j] = MIN
        
   return PRECIOS,RUTA

In [3]:
PRECIOS,RUTA = Precios(TARIFAS)  
#print(PRECIOS[0][6])

print("PRECIOS")
for i in range(len(TARIFAS)):
  print(PRECIOS[i])

print("\nRUTA")  
for i in range(len(TARIFAS)):
  print(RUTA[i]) 

PRECIOS
[9999, 5, 4, 3, 8, 8, 11]
[9999, 9999, 999, 2, 3, 8, 7]
[9999, 9999, 9999, 1, 6, 4, 7]
[9999, 9999, 9999, 9999, 5, 6, 9]
[9999, 9999, 9999, 9999, 9999, 999, 4]
[9999, 9999, 9999, 9999, 9999, 9999, 3]
[9999, 9999, 9999, 9999, 9999, 9999, 9999]

RUTA
['', 0, 0, 0, 1, 2, 5]
['', '', 1, 1, 1, 3, 4]
['', '', '', 2, 3, 2, 5]
['', '', '', '', 3, 3, 3]
['', '', '', '', '', 4, 4]
['', '', '', '', '', '', 5]
['', '', '', '', '', '', '']


In [0]:
# Solución con programación dinámica: Con ello dispondremos de todas las posibilidades para distintos caminos
# con una sola ejecución.
def calcular_ruta(RUTA, desde, hasta):
  if desde == hasta:
    #print("Ir a :" + str(desde))
    return desde 
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) +  ',' + str(RUTA[desde][hasta]) 

In [5]:
print("\nLa ruta es:")  
calcular_ruta(RUTA, 0,6) 


La ruta es:


'0,0,2,5'

In [0]:
# Problema de Asignación de tareas
COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

In [7]:
def valor(S,COSTES): # Devuelve el coste de la tarea indicada mediante los valores del vector de entrada S, 
                     # para cada agente según la posición
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i] 
  return VALOR
  
valor((0, 1, 2, 3),COSTES)

73

In [0]:
import itertools
def fuerza_bruta(COSTES):
   mejor_valor = 10e10
   mejor_solucion = ()

   for s in list(itertools.permutations(range(len(COSTES)))):
     #print(s,valor(s,COSTES))
     valor_tmp = valor(s,COSTES)
     if valor_tmp < mejor_valor:
       mejor_valor = valor_tmp
       mejor_solucion = s
   print("La mejor solucion es:",mejor_solucion," con valor:",mejor_valor)
  

In [0]:
#Coste inferior para soluciones parciales
#  (1,3,) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i]

  #Estimacion
  for i in range(len(COSTES)):
    if i not in S:
      VALOR += min([COSTES[j][i] for j in range(len(S),len(COSTES))])
  return VALOR

In [0]:
# Coste superior para soluciones parciales (No lo usaremos en ramificación y poda en esta ocasión)
def CS(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range(len(COSTES)):
    if i not in S:
      VALOR += max( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

In [0]:
#Genera tantos hijos como como posibilidades haya para la siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append({'s':NODO +(i,)    })
  return HIJOS

In [12]:
def ramificacion_y_poda(COSTES):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  { s:(1,2),CI:3,CS:5  }
   #print(COSTES)
   DIMENSION = len(COSTES)  
   MEJOR_SOLUCION=tuple( i for i in range(len(COSTES)))
   CotaSup = valor(MEJOR_SOLUCION,COSTES)
   #print("Cota Superior:", CotaSup)

   NODOS=[]
   NODOS.append({'s':(), 'ci':CI((),COSTES)})

   iteracion = 0
   while( len(NODOS) > 0):
     iteracion +=1

     nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']
     #print("Nodo prometedor:", nodo_prometedor)

     #Ramificacion
     #Se generan los hijos
     HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]

     #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
     NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION  ]
     if len(NODO_FINAL ) >0: 
       #print("\n********Soluciones:",  [x for x in HIJOS if len(x['s']) == DIMENSION  ] )
       if NODO_FINAL[0]['ci'] < CotaSup:
         CotaSup = NODO_FINAL[0]['ci']
         MEJOR_SOLUCION = NODO_FINAL
 
     #Poda
     HIJOS = [x for x in HIJOS if x['ci'] < CotaSup   ]

     #Añadimos los hijos 
     NODOS.extend(HIJOS) 

     #Eliminamos el nodo ramificado
     NODOS =  [  x for x in NODOS if x['s'] != nodo_prometedor    ]
   
   print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION  )
      

fuerza_bruta(COSTES)
print('RAMIFICACIÓN Y PODA:')
ramificacion_y_poda(COSTES)

La mejor solucion es: (0, 3, 1, 2)  con valor: 61
RAMIFICACIÓN Y PODA:
La solucion final es: [{'s': (0, 3, 1, 2), 'ci': 61}]  en  19  iteraciones  para dimension:  4


 - ¿Que complejidad tiene el algoritmo por fuerza bruta?
 - Generar matrices con valores aleatorios de mayores
dimensiones (5,6,7,…) y ejecutar ambos algoritmos.
 - ¿A partir de que dimensión el algoritmo por fuerza bruta
deja de ser una opción?
 - ¿Hay algún valor de la dimensión a partir de la cual el
algoritmo de ramificación y poda deja de ser una opción
válida?

 A la primera pregunta: $2 + n(2+n) => n^2$ es decir: $\Theta(n^2)$ sería la complejidad por fuerza bruta

In [0]:
# Matriz de dimensiones,
# 6:
import numpy as np
import random
COSTES_5 = np.random.randint(1, 99, (5, 5))
COSTES_6 = np.random.randint(1, 99, (6, 6))
COSTES_10 = np.random.randint(1, 99, (10, 10))
COSTES_11 = np.random.randint(1, 99, (11, 11))
COSTES_12 = np.random.randint(1, 99, (12, 12))
COSTES_13 = np.random.randint(1, 99, (13, 13))
COSTES_14 = np.random.randint(1, 99, (14, 14))
COSTES_15 = np.random.randint(1, 99, (15, 15))
COSTES_20 = np.random.randint(1, 99, (20, 20))



In [14]:
print('FUERZA BRUTA: ')
fuerza_bruta(COSTES_5)
print('RAMIFICACIÓN Y PODA: ')
ramificacion_y_poda(COSTES_5)

print('FUERZA BRUTA: ')
fuerza_bruta(COSTES_6)
print('RAMIFICACIÓN Y PODA: ')
ramificacion_y_poda(COSTES_6)

print('FUERZA BRUTA: ')
fuerza_bruta(COSTES_10)
print('RAMIFICACIÓN Y PODA: ')
ramificacion_y_poda(COSTES_10)

print('FUERZA BRUTA: KO para n=11')
#fuerza_bruta(COSTES_11)
print('RAMIFICACIÓN Y PODA: ')
ramificacion_y_poda(COSTES_11)

FUERZA BRUTA: 
La mejor solucion es: (0, 4, 1, 2, 3)  con valor: 125
RAMIFICACIÓN Y PODA: 
La solucion final es: [{'s': (0, 4, 1, 2, 3), 'ci': 125}]  en  19  iteraciones  para dimension:  5
FUERZA BRUTA: 
La mejor solucion es: (5, 4, 2, 1, 0, 3)  con valor: 112
RAMIFICACIÓN Y PODA: 
La solucion final es: [{'s': (5, 4, 2, 1, 0, 3), 'ci': 112}]  en  27  iteraciones  para dimension:  6
FUERZA BRUTA: 
La mejor solucion es: (4, 9, 8, 2, 5, 1, 7, 6, 0, 3)  con valor: 122
RAMIFICACIÓN Y PODA: 
La solucion final es: [{'s': (9, 3, 8, 2, 5, 6, 7, 4, 0, 1), 'ci': 153}]  en  2796  iteraciones  para dimension:  10
FUERZA BRUTA: KO para n=11
RAMIFICACIÓN Y PODA: 
La solucion final es: [{'s': (7, 10, 2, 1, 3, 9, 4, 6, 0, 8, 5), 'ci': 204}]  en  3159  iteraciones  para dimension:  11


Para n= 11 el algoritmo por fuerza bruta deja de ser una opción por error de ejecución por falta de RAM.

In [17]:

print('RAMIFICACIÓN Y PODA: ')
ramificacion_y_poda(COSTES_13)

RAMIFICACIÓN Y PODA: 
La solucion final es: [{'s': (6, 9, 4, 3, 12, 10, 11, 0, 8, 2, 5, 7, 1), 'ci': 184}]  en  97460  iteraciones  para dimension:  13


RAMIFICACIÓN Y PODA: 
La solucion final es: [{'s': (6, 9, 4, 3, 12, 10, 11, 0, 8, 2, 5, 7, 1), 'ci': 184}]  en  97460  iteraciones  para dimension:  13

In [19]:
print('RAMIFICACIÓN Y PODA: ')
ramificacion_y_poda(COSTES_14)
#print('RAMIFICACIÓN Y PODA: ')
#ramificacion_y_poda(COSTES_20)

RAMIFICACIÓN Y PODA: 


KeyboardInterrupt: ignored

En el caso de ramificación y poda, el límite es el tiempo de ejecución y no tanto la memoria .. a partir de 14 ya no pude 